# LIONS Analysis

Our project is to analyze federal cases.

In [1]:
from lions.parse import *
import pandas as pd
import numpy as np

In [2]:
case_sql, cases = gs_case(columns=['ID', 'DISTRICT', 'LEAD_CHARGE', 'CLASS'])
hist_sql, hist = gs_court_hist(columns=['CASEID', 'DISTRICT', 'COURT', 'APPEAL_TYPE', 'SENT_APPEAL', \
                                        'DISPOSITION', 'DISP_REASON1', 'DISP_REASON2', 'DISP_REASON3'])
court_judge_sql, court_judge = gs_court_judge(columns=['CASEID','JUDGEID','DISTRICT','DECISION'])
judge_sql, judge = gs_judge()

In [4]:
#cases = cases[:100000]
#hist = hist[:50000]
#court_judge = court_judge[:50000]

cases_df = pd.DataFrame(cases, columns = cases[0].keys())
del cases

hist_df = pd.DataFrame(hist, columns = hist[0].keys())
del hist

court_judge_df = pd.DataFrame(court_judge, columns = court_judge[0].keys())
del court_judge

judge_df = pd.DataFrame(judge, columns = judge[0].keys())
del judge

print(len(cases_df))
print(len(hist_df))

NameError: name 'cases' is not defined

In [6]:
cases_df = cases_df.sample(100000)

In [7]:
cases_hist_df = pd.merge(cases_df, hist_df, left_on = ['ID','DISTRICT'], right_on = ['CASEID','DISTRICT'], how = 'inner')
del cases_df
del hist_df
print("Done merging Cases DF and Hist DF")
print(len(cases_hist_df))

cases_hist_court_df = pd.merge(cases_hist_df, court_judge_df, left_on = ['CASEID','DISTRICT'], right_on = ['CASEID','DISTRICT'], how = 'inner')
del cases_hist_df
del court_judge_df
print("Done merging Cases Hist DF and Court Judge DF")
print(len(cases_hist_court_df))

cases_hist_court_judge_df = pd.merge(cases_hist_court_df, judge_df, left_on = ['JUDGEID','DISTRICT'], right_on = ['ID','DISTRICT'], how = 'inner')
del cases_hist_court_df
del judge_df
print("Done merging Cases Hist Court DF and Judge DF")
print(len(cases_hist_court_judge_df))


withdisp_df = cases_hist_court_judge_df[cases_hist_court_judge_df.DISPOSITION != '']
del cases_hist_court_judge_df
print(len(withdisp_df))

withdisp_reason_df = withdisp_df[withdisp_df.DISP_REASON1 != '']
print(len(withdisp_reason_df))

withoutdisp_reason_df = withdisp_df[withdisp_df.DISP_REASON1 == '']
print(len(withoutdisp_reason_df))

del withdisp_df

Done merging Cases DF and Hist DF
137280
Done merging Cases Hist DF and Court Judge DF
157156
Done merging Cases Hist Court DF and Judge DF
157115
152993
124802
28191


In [6]:
withdisp_reason_df.columns

Index(['ID_x', 'DISTRICT', 'LEAD_CHARGE', 'CLASS', 'CASEID', 'COURT',
       'APPEAL_TYPE', 'SENT_APPEAL', 'DISPOSITION', 'DISP_REASON1',
       'DISP_REASON2', 'DISP_REASON3', 'JUDGEID', 'DECISION', 'ID_y',
       'INITIALS', 'INIT_STAT', 'LAST_NAME', 'FIRST_NAME', 'COURT_ROOM',
       'TYPE', 'CREATE_DATE', 'CREATE_USER', 'UPDATE_DATE', 'UPDATE_USER'],
      dtype='object')

In [17]:
columns_to_drop = ['ID_x','CASEID','DISP_REASON1', 'DISP_REASON2',
       'DISP_REASON3', 'JUDGEID', 'DECISION', 'ID_y', 'INITIALS', 'INIT_STAT',
       'LAST_NAME', 'FIRST_NAME', 'COURT_ROOM', 'TYPE', 'CREATE_DATE',
       'CREATE_USER', 'UPDATE_DATE', 'UPDATE_USER']

disp_reasons = ['VACA','VARM','RDAP','RDRR','REVA','REVR','OFPO','GWDA','DEPO','LECI','WKEV','WTPR']


misconduct = []
disp1 = withdisp_reason_df['DISP_REASON1'].tolist()

for i in range(len(disp1)):
    if disp1[i] in disp_reasons: 
        misconduct.append(1)
    else:
        misconduct.append(0)


df_classify = withdisp_reason_df.drop(columns_to_drop, axis = 1)
df_to_classify = withoutdisp_reason_df.drop(columns_to_drop, axis=1)

df_classify.columns


Index(['DISTRICT', 'LEAD_CHARGE', 'CLASS', 'COURT', 'APPEAL_TYPE',
       'SENT_APPEAL', 'DISPOSITION'],
      dtype='object')

In [18]:
df_district = pd.get_dummies(df_classify['DISTRICT'], prefix = 'District')
df_charge = pd.get_dummies(df_classify['LEAD_CHARGE'], prefix = 'Charge')
df_class = pd.get_dummies(df_classify['CLASS'], prefix = 'Class')
df_court = pd.get_dummies(df_classify['COURT'], prefix = 'Court')
df_appeal = pd.get_dummies(df_classify['APPEAL_TYPE'], prefix = 'Appeal')
df_sent = pd.get_dummies(df_classify['SENT_APPEAL'], prefix = 'Sent')
df_disposition = pd.get_dummies(df_classify['DISPOSITION'], prefix = 'Disposition')

df_new = pd.concat([df_classify, df_district, df_charge, df_class, df_court, df_appeal, df_sent, df_disposition], axis=1)
del df_classify
del df_district
del df_charge
del df_class 
del df_court
del df_appeal
del df_sent
del df_disposition

df_new = df_new.drop(['DISTRICT','CLASS','LEAD_CHARGE', 'COURT', 'APPEAL_TYPE','SENT_APPEAL','DISPOSITION'], axis=1)
print("frist step done")


df_district = pd.get_dummies(df_to_classify['DISTRICT'], prefix = 'District')
df_charge = pd.get_dummies(df_to_classify['LEAD_CHARGE'], prefix = 'Charge')
df_class = pd.get_dummies(df_to_classify['CLASS'], prefix = 'Class')
df_court = pd.get_dummies(df_to_classify['COURT'], prefix = 'Court')
df_appeal = pd.get_dummies(df_to_classify['APPEAL_TYPE'], prefix = 'Appeal')
df_sent = pd.get_dummies(df_to_classify['SENT_APPEAL'], prefix = 'Sent')
df_disposition = pd.get_dummies(df_to_classify['DISPOSITION'], prefix = 'Disposition')

df_new_to_classify = pd.concat([df_to_classify, df_district, df_charge, df_class, df_court, df_appeal, df_sent, df_disposition], axis=1)
df_new_to_classify = df_new_to_classify.drop(['DISTRICT','CLASS','LEAD_CHARGE', 'COURT', 'APPEAL_TYPE','SENT_APPEAL','DISPOSITION'], axis=1)
print("second step done")

del df_to_classify
del df_district
del df_charge
del df_class 
del df_court
del df_appeal
del df_sent
del df_disposition

common_cols = list(set(df_new).intersection(df_new_to_classify))

df_new = df_new[common_cols]
df_new_to_classify = df_new_to_classify[common_cols]
df_new

frist step done
second step done


,District_GAN,District_OKE,Charge_18 :00922G1,Charge_18 :02113D,District_MOE,District_WVN,Charge_18 :00844i,Charge_26 :07201,Charge_21 :00841/846,District_ARE,...,Disposition_DA,District_ALM,District_NYW,Charge_18 :02113a,Charge_21 :00841a,District_GU,Charge_49 :46502,Charge_18 :00500,Charge_18 :01512B3,District_TNM
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
#generate our train and test dataset
m = 100000

df_to_be_classified = df_new_to_classify

df_train = df_new.iloc[:m]
df_test = df_new.iloc[m:]
list_train = misconduct[:m]
list_test = misconduct[m:]

df_train

,District_GAN,District_OKE,Charge_18 :00922G1,Charge_18 :02113D,District_MOE,District_WVN,Charge_18 :00844i,Charge_26 :07201,Charge_21 :00841/846,District_ARE,...,Disposition_DA,District_ALM,District_NYW,Charge_18 :02113a,Charge_21 :00841a,District_GU,Charge_49 :46502,Charge_18 :00500,Charge_18 :01512B3,District_TNM
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# from sklearn.neighbors import KNeighborsClassifier
# import sklearn.model_selection as cross_validation

# #build our knn model and print the accuracy score
# knn = KNeighborsClassifier(n_neighbors=10)
# knn.fit(df_train, list_train)

# #print the accuracy score
# print('KNeighbor accuracy on test data: ', knn.score(df_test, list_test))
# print('KNeighbor accuracy on training data: ', knn.score(df_train, list_train))

# #accuracy score from crossvalidation
# scores = cross_validation.cross_val_score(knn, df_new, misconduct, cv=5)
# print("KNN Accuracy from crossvalidation:: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

KNeighbor accuracy on test data:  0.958860892256316
KNeighbor accuracy on training data:  0.968
KNN Accuracy from crossvalidation:: 0.96 (+/- 0.01)


In [21]:
from sklearn import tree
import sklearn.model_selection as cross_validation

#build our decision trees model
dtc = tree.DecisionTreeClassifier()
dtc.fit(df_train, list_train)

#print the accuracy score
print('DT accuracy on test data: ', dtc.score(df_test, list_test))
print('DT accuracy on training data: ', dtc.score(df_train, list_train))

#accuracy score from crossvalidation
scores = cross_validation.cross_val_score(dtc, df_new, misconduct, cv=5)
print("DT Accuracy from crossvalidation: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

DT accuracy on test data:  0.9948794452060318
DT accuracy on training data:  0.99817
DT Accuracy from crossvalidation: 1.00 (+/- 0.00)


In [22]:
from sklearn.linear_model import LogisticRegression

#build our logistic regression model
lgr = LogisticRegression(solver='lbfgs', multi_class='multinomial')
lgr.fit(df_train, list_train)

#print the accuracy score
print("Accuracy of logistic regression test set:", lgr.score(df_test, list_test))
print("Accuracy of logistic regression train set:", lgr.score(df_train, list_train))

#accuracy score from crossvalidation
scores = cross_validation.cross_val_score(lgr, df_new, misconduct, cv=5)
print("Logistic Regression Accuracy from crossvalidation: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy of logistic regression test set: 0.994798806547859
Accuracy of logistic regression train set: 0.9962
Logistic Regression Accuracy from crossvalidation: 1.00 (+/- 0.00)


In [23]:
#Apply the classifier to the dataset withoutdisp_reason_df

df_to_be_classified

,District_GAN,District_OKE,Charge_18 :00922G1,Charge_18 :02113D,District_MOE,District_WVN,Charge_18 :00844i,Charge_26 :07201,Charge_21 :00841/846,District_ARE,...,Disposition_DA,District_ALM,District_NYW,Charge_18 :02113a,Charge_21 :00841a,District_GU,Charge_49 :46502,Charge_18 :00500,Charge_18 :01512B3,District_TNM
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
df_prediction_knn = lgr.predict(df_to_be_classified)

df_results = withoutdisp_reason_df.copy()
df_results['misconduct'] = df_prediction_knn
df_misconduct1 = df_results[df_results['misconduct'] == 1]


df_result = withdisp_reason_df.copy()
df_result['misconduct'] = misconduct
df_misconduct2 = df_result[df_result['misconduct'] == 1]


df_misconduct = pd.concat([df_misconduct1,df_misconduct2],axis=0)
df_misconduct['DISTRICT'].value_counts()

CAS    96
ILN    72
NYS    50
WVS    47
DC     39
CAC    35
MIE    27
PR     27
AZ     27
TXW    23
MT     21
MIW    21
OR     18
MSS    15
MOE    15
TXS    15
NM     14
FLM    14
OHN    13
NYW    12
TNW    12
NJ     12
PAE    12
KYE    12
ILS    11
FLS    10
NCW    10
WAW    10
CAE     9
OHS     9
       ..
TXN     5
FLN     5
KYW     5
IAN     5
NE      4
SD      4
VAW     4
TXE     4
WY      4
NYN     3
GAM     3
INN     3
MA      3
CO      3
INS     3
ARE     3
GAN     3
NH      3
PAW     2
CT      2
VI      2
OKE     2
ND      2
HI      1
ALS     1
IAS     1
TNM     1
ID      1
GAS     1
ALM     1
Name: DISTRICT, Length: 80, dtype: int64